In [6]:
%load_ext autoreload
%autoreload 2

import sys
if '..' not in sys.path:
    sys.path.append('..')
from notebooks.results import load_results, classification, clusterization, to_latex_table, GremDataFrame, include_pivot_index, groups_to_latex_table
import re
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
GREMVECS = ['BigramMorphTagVectorizer100', 'BigramMorphTagVectorizer370', 'FullMorphTagVectorizer', 'MorphTagVectorizer', 'StyloMetrix']
BOWVECS = ['CountVectorizer1000', 'CountVectorizer5000', 'TfidfVectorizer1000', 'TfidfVectorizer5000', 'CountTfidf1000']
SEMVECS = ['HerbertFT', 'HerbertFrozen', 'RoBERTaFT', 'RoBERTaFrozen', 'DPEBPVectorizer']
GREMSEMVECS = ['GremBERT', 'PanGremBERT', 'FrozenGremBERT', 'FrozenPanGremBERT']

def extract_letters(s: str) -> str:
    return re.findall(r'[a-zA-Z]+', s)[0]

def fix_dataset_name(s: str) -> str:
    return {
        'TweeterCyberbullying': 'TwitterCyberbullying'
    }.get(s, s)

def fix_vectorizer_name(s: str) -> str:
    return {
        'DPEBPVectorizer100Avg': 'DPEBPVectorizer',
        'SpacyMorphTagVectorizer': 'MorphTagVectorizer',
    }.get(s, s)

def assign_vectorizer_type(name: str) -> str:
    if name in BOWVECS:
        return 'zliczający'
    if name in SEMVECS:
        return 'semantyczny'
    if name in GREMVECS:
        return 'gramatyczny'
    if name in GREMSEMVECS:
        return 'gramatyczno-semantyczny'
    return 'nieznany'

def vector_type_sorter(data: tuple[str, pd.DataFrame]) -> int:
    name = data[0]
    if name == 'zliczający':
        return 1
    if name == 'gramatyczny':
        return 2
    if name == 'semantyczny':
        return 3
    if name == 'gramatyczno-semantyczny':
        return 4
    return 5

In [8]:
results_df = GremDataFrame(load_results('../results/'))
results_df['dataset'] = results_df['dataset'].apply(fix_dataset_name)
results_df['vectorizer'] = results_df['vectorizer'].apply(fix_vectorizer_name)
results_df['base_head_model'] = results_df['params_name'].apply(extract_letters)
results_df['vector_type'] = results_df['vectorizer'].apply(assign_vectorizer_type)
results_df = GremDataFrame(results_df[results_df['vector_type'] != 'nieznany'])
print(len(results_df))
results_df.head()

1596


,dataset,datacleaner,vectorizer,params_name,accuracy,f1_score,recall,precision,silhouette,davies_bouldin,calinski_harabasz,bcubed_precission,bcubed_recall,bcubed_f1,base_head_model,vector_type
0,TwitterCyberbullying,DummyDatacleaner,MorphTagVectorizer,MLP1,0.915423,0.477922,0.5,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,MLP,gramatyczny
1,TwitterCyberbullying,DummyDatacleaner,MorphTagVectorizer,RandomForest1,0.915423,0.477922,0.5,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,RandomForest,gramatyczny
2,TwitterCyberbullying,DummyDatacleaner,MorphTagVectorizer,LogisticRegression1,0.915423,0.477922,0.5,0.457711,NaN,NaN,NaN,NaN,NaN,NaN,LogisticRegression,gramatyczny
3,TwitterCyberbullying,DummyDatacleaner,MorphTagVectorizer,KMeans1.0,NaN,NaN,NaN,NaN,0.086742,3.113713,889.408505,0.845114,0.501334,0.629336,KMeans,gramatyczny
4,TwitterCyberbullying,DummyDatacleaner,MorphTagVectorizer,DBSCAN1,NaN,NaN,NaN,NaN,0.213195,2.498378,56.533770,0.844912,0.928915,0.884924,DBSCAN,gramatyczny


In [23]:
results_df.to_parquet('../witek/results.parquet')

# Klasyfikacja

In [24]:
df = (
    results_df
    .classification()
    .dataset('Classics5Authors35Books')
    .data_cleaner('ProperNamesMasker')
    .sort_values('f1_score', ascending=False)
    .groupby('vectorizer')
    .first()
)
df = include_pivot_index(df, 'vectorizer')
gdf = df.groupby('vector_type')[['vectorizer', 'recall', 'precision', 'accuracy', 'f1_score']]

print(groups_to_latex_table(
    gdf,
    groups_sort_key=vector_type_sorter,
    bold_labels=['f1_score', 'accuracy'],
    column_names=['typ wektorów', 'wektoryzator', 'pełność (recall)', 'precyzja', 'dokładność', 'miara f1'],
))

\begin{table}[H]
	\centering
	\caption{}
	\resizebox{\textwidth}{!}{
	\begin{tabular}{||c|c|c|c|c|c||}
		\hline
		\textbf{typ wektorów} & \textbf{wektoryzator} & \textbf{pełność (recall)} & \textbf{precyzja} & \textbf{dokładność} & \textbf{miara f1} \\
		\hline
		\multirow{5}{*}{zliczający} & CountTfidf1000 & 0.940 & 0.914 & \textbf{0.948} & \textbf{0.925} \\
		 & CountVectorizer1000 & 0.909 & 0.877 & 0.922 & 0.888 \\
		 & CountVectorizer5000 & 0.937 & 0.902 & 0.940 & 0.914 \\
		 & TfidfVectorizer1000 & 0.926 & 0.888 & 0.932 & 0.903 \\
		 & TfidfVectorizer5000 & 0.904 & 0.882 & 0.931 & 0.889 \\
\hline		\multirow{5}{*}{gramatyczny} & BigramMorphTagVectorizer100 & 0.872 & 0.881 & 0.880 & 0.872 \\
		 & BigramMorphTagVectorizer370 & 0.898 & 0.890 & \textbf{0.902} & \textbf{0.891} \\
		 & FullMorphTagVectorizer & 0.785 & 0.755 & 0.798 & 0.748 \\
		 & MorphTagVectorizer & 0.749 & 0.713 & 0.726 & 0.699 \\
		 & StyloMetrix & 0.786 & 0.779 & 0.816 & 0.763 \\
\hline		\multirow{5}{*}{semantyczny}

In [25]:
df = (
    results_df
    .classification()
    .dataset('Classics5Authors35Books')
    .data_cleaner('DummyDatacleaner')
    .sort_values('f1_score', ascending=False)
    .groupby('vectorizer')
    .first()
    [['accuracy', 'f1_score', 'recall', 'precision', 'base_head_model']]
)
df = include_pivot_index(df, 'wektoryzator')
print(to_latex_table(
    df,
    column_names=['wektoryzator', 'dokładność', 'miara f1', 'pełność (recall)', 'precyzja', 'model UM'],
    bold_labels=['f1_score', 'accuracy', 'recall', 'precision'],
))
df.style.highlight_max(color = '#666666', axis = 0)

\begin{table}[H]
	\centering
	\caption{}
	\resizebox{\textwidth}{!}{
	\begin{tabular}{||c|c|c|c|c|c||}
		\hline
		\textbf{wektoryzator} & \textbf{dokładność} & \textbf{miara f1} & \textbf{pełność (recall)} & \textbf{precyzja} & \textbf{model UM} \\
		\hline
		BigramMorphTagVectorizer100 & 0.890 & 0.880 & 0.883 & 0.882 & MLP \\
		BigramMorphTagVectorizer370 & 0.898 & 0.887 & 0.889 & 0.888 & MLP \\
		CountTfidf1000 & \textbf{0.934} & \textbf{0.908} & \textbf{0.933} & \textbf{0.892} & MLP \\
		CountVectorizer1000 & 0.926 & 0.897 & 0.923 & 0.881 & MLP \\
		CountVectorizer5000 & 0.925 & 0.895 & 0.914 & 0.888 & LogisticRegression \\
		DPEBPVectorizer & 0.638 & 0.593 & 0.675 & 0.652 & MLP \\
		FrozenGremBERT & 0.721 & 0.664 & 0.704 & 0.679 & MLP \\
		FrozenPanGremBERT & 0.844 & 0.813 & 0.827 & 0.810 & MLP \\
		FullMorphTagVectorizer & 0.778 & 0.725 & 0.756 & 0.740 & MLP \\
		GremBERT & 0.805 & 0.751 & 0.776 & 0.745 & MLP \\
		HerbertFT & 0.788 & 0.732 & 0.769 & 0.725 & LogisticRegression \\
	

,wektoryzator,accuracy,f1_score,recall,precision,base_head_model
vectorizer,,,,,,
BigramMorphTagVectorizer100,BigramMorphTagVectorizer100,0.889595,0.879816,0.882668,0.882232,MLP
BigramMorphTagVectorizer370,BigramMorphTagVectorizer370,0.897538,0.887260,0.889314,0.888123,MLP
CountTfidf1000,CountTfidf1000,0.934075,0.908142,0.933483,0.891808,MLP
CountVectorizer1000,CountVectorizer1000,0.926132,0.897451,0.922941,0.880877,MLP
CountVectorizer5000,CountVectorizer5000,0.925338,0.894927,0.913577,0.888191,LogisticRegression
DPEBPVectorizer,DPEBPVectorizer,0.637808,0.593334,0.674686,0.651821,MLP
FrozenGremBERT,FrozenGremBERT,0.721207,0.663970,0.703689,0.679403,MLP
FrozenPanGremBERT,FrozenPanGremBERT,0.844321,0.812815,0.826891,0.810132,MLP
FullMorphTagVectorizer,FullMorphTagVectorizer,0.778396,0.725368,0.756164,0.740067,MLP


# Grupowanie

In [30]:
df = (
    results_df
    .clusterization()
    .dataset('Classics5Authors35Books')
    .data_cleaner('DummyDatacleaner')
    .sort_values('bcubed_f1', ascending=False)
    .groupby('vectorizer')
    .first()
)
df = include_pivot_index(df, 'vectorizer')
gdf = df.groupby('vector_type')[['vectorizer', 'base_head_model', 'silhouette', 'bcubed_f1']]
print(groups_to_latex_table(
    gdf,
    groups_sort_key=vector_type_sorter,
    column_names=['typ wektorów', 'wektoryzator', 'sposób grupowania', 'indeks silhouette', 'bcubed f1'],
    bold_labels=['bcubed_f1', 'silhouette'],
))

# df = (
#     results_df
#     .clusterization()
#     .dataset('Classics5Authors35Books')
#     .data_cleaner('DummyDatacleaner')
#     .sort_values('bcubed_f1', ascending=False)
#     .groupby('vectorizer')
#     .first()
# )
# df = include_pivot_index(df, 'vectorizer')
# gdf = df.groupby('vector_type')[['vectorizer', 'silhouette', 'bcubed_recall', 'bcubed_precission', 'bcubed_f1']]
# print(groups_to_latex_table(
#     gdf,
#     groups_sort_key=vector_type_sorter,
#     column_names=['typ wektorów', 'wektoryzator', 'indeks silhouette', 'pełność bcubed (recall)', 'precyzja bcubed', 'bcubed f1'],
#     bold_labels=['bcubed_f1', 'silhouette'],
# ))

\begin{table}[H]
	\centering
	\caption{}
	\resizebox{\textwidth}{!}{
	\begin{tabular}{||c|c|c|c|c||}
		\hline
		\textbf{typ wektorów} & \textbf{wektoryzator} & \textbf{sposób grupowania} & \textbf{indeks silhouette} & \textbf{bcubed f1} \\
		\hline
		\multirow{5}{*}{zliczający} & CountTfidf1000 & DBSCAN & -1.000 & 0.407 \\
		 & CountVectorizer1000 & DBSCAN & -1.000 & 0.407 \\
		 & CountVectorizer5000 & DBSCAN & -1.000 & 0.407 \\
		 & TfidfVectorizer1000 & KMeans & \textbf{0.024} & 0.459 \\
		 & TfidfVectorizer5000 & KMeans & 0.005 & \textbf{0.531} \\
\hline		\multirow{5}{*}{gramatyczny} & BigramMorphTagVectorizer100 & DBSCAN & \textbf{0.515} & 0.407 \\
		 & BigramMorphTagVectorizer370 & DBSCAN & 0.512 & 0.407 \\
		 & FullMorphTagVectorizer & DBSCAN & 0.462 & 0.407 \\
		 & MorphTagVectorizer & DBSCAN & -1.000 & 0.407 \\
		 & StyloMetrix & KMeans & 0.157 & \textbf{0.415} \\
\hline		\multirow{5}{*}{semantyczny} & DPEBPVectorizer & DBSCAN & 0.472 & 0.409 \\
		 & HerbertFT & KMeans & 0.754 

In [29]:
df = (
    results_df
    .clusterization()
    .dataset('Classics5Authors35Books')
    .data_cleaner('DummyDatacleaner')
    .sort_values('bcubed_f1', ascending=False)
    .groupby('vectorizer')
    .first()
)
df = include_pivot_index(df, 'vectorizer')
gdf = df[['vectorizer', 'silhouette', 'bcubed_recall', 'bcubed_precission', 'bcubed_f1', 'base_head_model']]
df = include_pivot_index(gdf, 'vector_type')
df.style.highlight_max(color = '#666666', axis = 0)

d:\HDD\Studia\Mgr\grembedding\notebooks\..\notebooks\results.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df.index


,vector_type,vectorizer,silhouette,bcubed_recall,bcubed_precission,bcubed_f1,base_head_model
vectorizer,,,,,,,
BigramMorphTagVectorizer100,BigramMorphTagVectorizer100,BigramMorphTagVectorizer100,0.514989,0.999151,0.255663,0.407146,DBSCAN
BigramMorphTagVectorizer370,BigramMorphTagVectorizer370,BigramMorphTagVectorizer370,0.512334,0.999151,0.255663,0.407146,DBSCAN
CountTfidf1000,CountTfidf1000,CountTfidf1000,-1.000000,1.000000,0.255567,0.407095,DBSCAN
CountVectorizer1000,CountVectorizer1000,CountVectorizer1000,-1.000000,1.000000,0.255567,0.407095,DBSCAN
CountVectorizer5000,CountVectorizer5000,CountVectorizer5000,-1.000000,1.000000,0.255567,0.407095,DBSCAN
DPEBPVectorizer,DPEBPVectorizer,DPEBPVectorizer,0.472485,0.994934,0.257092,0.408601,DBSCAN
FrozenGremBERT,FrozenGremBERT,FrozenGremBERT,-1.000000,1.000000,0.255567,0.407095,DBSCAN
FrozenPanGremBERT,FrozenPanGremBERT,FrozenPanGremBERT,-1.000000,1.000000,0.255567,0.407095,DBSCAN
FullMorphTagVectorizer,FullMorphTagVectorizer,FullMorphTagVectorizer,0.462459,0.999151,0.255663,0.407146,DBSCAN


In [43]:
pd.options.display.float_format = '{:,.3f}'.format
vecs = list(results_df['vectorizer'].unique())
vecs.remove('HerbertVectorizer')

df = (
    results_df
    .classification()
    .vectorizer(vecs)
    # .data_cleaner('ProperNamesMasker')
    .sort_values('f1_score', ascending=False)
    .drop_duplicates(subset=['dataset', 'vectorizer'])
    .drop(columns=list(results_df.columns[6:-1]) + ['datacleaner', 'params_name'])
    .pivot(index='vectorizer', columns='dataset', values='f1_score')
    .style.highlight_max(color = '#666666', axis = 0)
)
# df = include_pivot_index(df, 'zbiór danych')
df

dataset,Classics5Authors35Books,EroticVsOthers,OldNewspapers,PrusVsSienkiewicz,StarWarsFanfic,StarWarsFanficMedium,StarWarsFanficShort,TwitterCyberbullying,WritingStyle
vectorizer,,,,,,,,,
BigramMorphTagVectorizer100,0.879816,0.596767,0.089154,0.750679,0.963278,0.818753,0.686301,0.500348,0.536801
BigramMorphTagVectorizer370,0.891134,0.600580,0.116297,0.796445,0.900966,0.850944,0.709485,0.615311,0.537584
CountTfidf1000,0.924524,0.646749,0.131315,0.883495,0.987759,0.883220,0.738083,0.632196,0.663568
CountVectorizer1000,0.897451,0.646804,0.128188,0.883495,0.975478,0.876556,0.738083,0.626262,0.656544
CountVectorizer5000,0.914083,0.655272,0.164850,0.899799,0.987759,0.882460,0.760631,0.708266,0.671502
DPEBPVectorizer,0.593334,0.615427,0.151467,0.706392,0.938943,0.864065,0.747152,0.500348,0.561997
FrozenGremBERT,0.663970,0.615279,0.193199,0.763844,0.902206,0.870549,0.806167,0.609427,0.589378
FrozenPanGremBERT,0.812815,0.616076,0.184294,0.687360,0.902206,0.889939,0.789929,0.548467,0.597499
FullMorphTagVectorizer,0.748191,0.604915,0.119493,0.825440,0.926655,0.799182,0.720949,0.632452,0.573628
